# City similarity [ Bangkok - Kyoto ]

## Import zone

In [1]:
import gensim
import os
import collections
import smart_open
import random
import pickle
import pandas as pd
import copy
import re

C:\Users\sociocom\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Parameter zone

In [2]:
path_play = r'C:\Users\sociocom\Desktop\Crawler\backup\playing'

In [3]:
grid_size = 10
diff = 0.1/grid_size

In [4]:
bkk_lat = 13.685780
bkk_long = 100.484605

In [5]:
kyt_lat = 34.931461
kyt_long = 135.685300

In [6]:
#merge to 10 color
color_idx = {}
idx_color = {}

## Function zone

In [7]:
def save_obj(file, obj):
    with open(file, 'wb') as f:
        pickle.dump(obj, f)

def load_obj(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [8]:
def clean_place(df, name):
    return df[df['place']!=name]

In [9]:
def separate_loc(df, flat, flong):
    df['ex_lat'] = df['ex_lat'].apply(lambda x: float(x))
    df['ex_long'] = df['ex_long'].apply(lambda x: float(x))
    data = []
    lat = flat
    long = flong
    for i in range(grid_size):
        for j in range(grid_size):
            data.append(df[(df['ex_lat'] >= lat) & (df['ex_lat'] < lat + diff) & (df['ex_long'] >= long) & (df['ex_long'] < long + diff)])
            idx = []
            idx.append((lat, long))
            idx.append((lat, long+diff))
            #print(lat," ",long)
            lat += diff
            idx.append((lat,long))
            idx.append((lat, long+diff))
            u = i*10 + j
            idx.append(str(u))
            idx_color[str(u)] = idx
        long += diff
        lat = flat
    return data

In [10]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="utf-8") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [11]:
def in_tuple(tp, x):
    return [z for y,z in tp if y == x]

## Data zone

### clean data

In [22]:
df_bkk1 = load_obj(path_play + r"\translated_bkk_tweet1")
df_bkk2 = load_obj(path_play + r"\translated_bkk_tweet2")

In [23]:
df_bkk = pd.concat([df_bkk1, df_bkk2])

In [24]:
df_bkk.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,68120,68120,68120,68120,68120,68120,68120,68120,68120,68120,68120,68120,68120,68120
unique,10746,10772,68120,59200,64013,2443,2426,149,104,102,16486,9040,9046,2
top,453950816,xiahjaashop,1002211513355128832,I do not know what to do.,2018-06-08 06:27:52,-1,-1,Thailand,"100.528375, 13.72951","100.541027, 13.749708",-1,-1,-1,th
freq,442,442,1,861,5,62585,62585,4758,5172,5172,50536,49779,49779,56098


In [25]:
df_bkk = clean_place(df_bkk, "Myanmar")
df_bkk = clean_place(df_bkk, "Thailand")
df_bkk = clean_place(df_bkk, "Bangkok")

In [26]:
df_bkk.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,58286,58286,58286,58286,58286,58286,58286,58286,58286,58286,58286,58286,58286,58286
unique,8886,8902,58286,50273,55311,2311,2295,146,101,99,14574,7864,7869,2
top,453950816,xiahjaashop,1002211513355128832,I do not know what to do.,2018-06-08 06:27:52,-1,-1,Pathum Wan,"100.528375, 13.72951","100.541027, 13.749708",-1,-1,-1,th
freq,442,442,1,791,5,53142,53142,3256,5172,5172,43171,42603,42603,49950


In [27]:
len(df_bkk[(df_bkk["ex_lat"]!= -1) & (df_bkk["tweet_reply_id"]!= "-1")])

15115

In [28]:
df_bkk.reset_index(inplace=True)

In [29]:
df_bkk["text"][1]

'Walk through (@Sam Market Market (Wat Phra Kaew Market) in Bang Rak, Bangkok) https://t.co/7TsvbPXdGE'

In [30]:
for i in range(len(df_bkk)):
    if (df_bkk["ex_lat"][i] == -1) | (df_bkk["tweet_reply_username"][i] == -1):
        continue
    idx_reply = df_bkk["text"][i].find(df_bkk["tweet_reply_username"][i])-1
    leng = len(df_bkk["tweet_reply_username"][i])+1
    df_bkk.at[i, "text"] = df_bkk["text"][i][:idx_reply] + " <MENTION> " + df_bkk["text"][i][idx_reply+leng:]

In [31]:
df_bkk.head()

,index,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,0,137552452,ExpatDanBKK,1001726470962769920,"<MENTION> Hmmm, I suspect Bill Heinecke woul...",2018-05-30 07:26:16,13.742844,100.5682602,Vadhana,"100.55007, 13.732979","100.570733, 13.74842",1001725702192824321,710742220681293824,David_Aamez,en
1,1,372674276,NeungBellerin,1001726500725456897,Walk through (@Sam Market Market (Wat Phra Kae...,2018-05-30 07:26:23,13.724922,100.52291,Suriya Wong,"100.516253, 13.72236","100.536283, 13.730574",-1,-1,-1,th
2,2,1030302462,opalopongpang,1001726547160690689,I'm at Penguin Eat Shabu in Vadhana https://t....,2018-05-30 07:26:34,13.7388373,100.56067598,Vadhana,"100.55007, 13.732979","100.570733, 13.74842",-1,-1,-1,en
3,3,953277866661199872,Poppysuphannee,1001726564550168576,And it's not going to be a school ride. I do n...,2018-05-30 07:26:38,-1,-1,Frown,"100.469033, 13.773245","100.49389, 13.789877",-1,-1,-1,th
4,4,399872095,StoneSoupBKK,1001726646599041024,<MENTION> F@$king morons.,2018-05-30 07:26:58,-1,-1,Phra Khanong Nuea,"100.584345, 13.704628","100.606727, 13.729193",1001701922074701824,774174489458331649,TheMossadIL,en


In [32]:
bkk = separate_loc(df_bkk, bkk_lat, bkk_long)

In [33]:
idx_color["3"]

[(13.715779999999999, 100.484605),
 (13.715779999999999, 100.494605),
 (13.725779999999999, 100.484605),
 (13.725779999999999, 100.494605),
 '3']

In [34]:
count = 0
valid = 0
for i in range(len(bkk)):
    count += len(bkk[i])
    if len(bkk[i])>20:
        valid += 1
    print("#tweets in area",i,":" ,len(bkk[i]))
print(count)
print(valid)

#tweets in area 0 : 0
#tweets in area 1 : 9
#tweets in area 2 : 6
#tweets in area 3 : 27
#tweets in area 4 : 3
#tweets in area 5 : 18
#tweets in area 6 : 100
#tweets in area 7 : 30
#tweets in area 8 : 24
#tweets in area 9 : 14
#tweets in area 10 : 2
#tweets in area 11 : 7
#tweets in area 12 : 5
#tweets in area 13 : 32
#tweets in area 14 : 11
#tweets in area 15 : 15
#tweets in area 16 : 81
#tweets in area 17 : 146
#tweets in area 18 : 23
#tweets in area 19 : 21
#tweets in area 20 : 0
#tweets in area 21 : 8
#tweets in area 22 : 3
#tweets in area 23 : 24
#tweets in area 24 : 15
#tweets in area 25 : 75
#tweets in area 26 : 8
#tweets in area 27 : 6
#tweets in area 28 : 9
#tweets in area 29 : 6
#tweets in area 30 : 1
#tweets in area 31 : 5
#tweets in area 32 : 5
#tweets in area 33 : 41
#tweets in area 34 : 28
#tweets in area 35 : 42
#tweets in area 36 : 11
#tweets in area 37 : 5
#tweets in area 38 : 6
#tweets in area 39 : 11
#tweets in area 40 : 1
#tweets in area 41 : 1
#tweets in area 42 : 

In [35]:
idx_color["56"] #Siam

[(13.745779999999998, 100.53460500000003),
 (13.745779999999998, 100.54460500000003),
 (13.755779999999998, 100.53460500000003),
 (13.755779999999998, 100.54460500000003),
 '56']

### prepare data

In [36]:
bkk[1]['text']

10536    I'm at Anantara Shuttle Boat in Thonburi, Bang...
14179    I'm at SKYLINE https://t.co/Ab6620Sg <MENTION>...
15026    I'm at AvaniFit in Bangkok https://t.co/jRiIY2...
16634    I'm at Bukkhalo Intersection in Thon Buri, Ban...
18136    I'm at Riverside Plaza in Thon Buri, Bangkok h...
18212    I'm at ThaiExpress in Thon Buri, Bangkok https...
18350    I'm at Attitude https://t.co/QNVi9y0m <MENTION...
25892    I'm at Riverside Plaza in Thon Buri, Bangkok h...
55179    Just posted a photo @ ... https://t.co/DhA9hlE...
Name: text, dtype: object

In [38]:
#pattern = re.compile(r"(I'm at [A-Za-z0-9 ,()'-]+ <URL>.)+")

In [39]:
bkk_str = []
for y in bkk:
    doc = ""
    for x in y['text']:
        if "https://" in x.lower():
            idx = (x.lower().find("https://"))-1
            x = x[:idx] + " <URL>"
        if "I'm at " in x and "<URL>" in x:
            idx_i = (x.find("I'm at "))
            idx_url = (x.find("<URL>"))+5
            x = x[:idx_i]+x[idx_url:]+"<CHECK_IN>"
        doc += x + ". "
    doc += " \n"
    bkk_str.append(doc)

In [40]:
#for y in bkk_str:
#    pattern2.sub("<CHECK_IN>", y)
#    print(y)

In [41]:
print(bkk_str[25])

<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Near Wat Lung Na Yi (Yaowarat in Samphanthawong, Bangkok) <URL>. I'm new here (@Yellow Shopping Center in Bangkok, Thailand) <URL>. <CHECK_IN>. <CHECK_IN>. I'm sorry for the dela <MENTION> 'm sorry for the delay.. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. I'm not sure if this is the right place for yo <MENTION> 'm not sure if this is the right place for you.. But I eat rice (@ Duck Noodle in Samphanthawong, Bangkok) <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. ((((((Https https https https https https https https https https https https https https https https https https https https https https htt <MENTION> (((((Https https https https https https https https https https https https https https https https https https https https https https https. <CHECK_

In [42]:
f = open(path_play + "/bkk_demo_train2","w",encoding="utf-8")
#file = open(path + "bkk_demo_test","w",encoding="utf-8")
for x in bkk_str:
    f.write(x)
    print(x)
f.close()

 

<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Just posted a photo @ ... <URL>.  

<CHECK_IN>. <CHECK_IN>. The first dental implant (@ dentist dentist) <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>.  

This #job might be a great fit for you: HR Associate Manager - Thailand - <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Can you recommend anyone for this #job? Data Analyst Manager - <URL>. This #job might be a great fit for you: Site E2E Operations Leader - <URL>. Want to work at The Procter &amp; Gamble Company? We're #hiring in #Bangkok! Click for details: <URL>. Want to work in #Bangkok? View our latest opening: <URL>. But I still can not. <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Stress (@ home in Bangkok) <URL>. <CHECK_IN>. Work with a gang with an apprentice. Same model (@ Bester Studio in Bangkok) <URL>. If you're looking for work in #Bangkok, check ou

## Doc2Vec

In [43]:
train_corpus = list(read_corpus(path_play + "/bkk_demo_train2"))

In [44]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=80, min_count=2, epochs=100, dm=0)

In [45]:
model.build_vocab(train_corpus)

In [46]:
#model.wv.vocab["สยาม"].count

In [47]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 1 s


In [48]:
ranks = []
second_ranks = []
all_sims = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    all_sims.append(sims)
    
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [49]:
print(ranks,"\n")
print(ranks.count(0))

[33, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 16, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 31, 20, 3, 0, 0, 0, 0, 0, 0, 0, 83, 0, 0, 3, 0, 0, 0, 0, 0, 1, 1, 99, 0, 0, 0, 0, 0, 0, 0, 0, 10, 1, 0, 0, 0, 0, 0, 0, 0] 

82


In [50]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (99): «virginstar listen to the song and tell feeling url check_in dpcynl cattshirt cattshirt catradio chayanit patchayanit url just began running workout using endomondo peptalk me now url virginhitz and listen to the song feeling url virginhitz feel free to listen and tell feeling url check_in just began running workout using endomondo peptalk me now url just began running workout using endomondo peptalk me now url just finished running km in with endomondo endorphins url»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dbow,d80,n5,mc2,s0.001,t3):

MOST (99, 0.9754493236541748): «virginstar listen to the song and tell feeling url check_in dpcynl cattshirt cattshirt catradio chayanit patchayanit url just began running workout using endomondo peptalk me now url virginhitz and listen to the song feeling url virginhitz feel free to listen and tell feeling url check_in just began running workout using endomondo peptalk me now url just began running workout using endomondo peptalk me n

## Merge area

In [51]:
boundary = 10

In [52]:
def merge_area(threshold, idx_color):
    for i in range(len(idx_color)):
        w = i-1 if (i-1)%10 != 9 else -1
        e = i+1 if ((i+1)%10 != 0) and (i+1 < len(idx_color)) else -1
        n = i-10 
        s = i+10 if i+10 < len(idx_color) else -1
        if len(bkk[i])<boundary:
            idx_color[str(i)][4] = "-1"
        else:
            if w>0 and len(bkk[w])>=boundary:
                similarw = (in_tuple(all_sims[i], w)[0] + in_tuple(all_sims[w], i)[0])/2
                if similarw > threshold:
                    idx_color[str(w)][4] = idx_color[str(i)][4]
            if e>0 and len(bkk[e])>=boundary:
                similare = (in_tuple(all_sims[i], e)[0] + in_tuple(all_sims[e], i)[0])/2
                if similare > threshold:
                    idx_color[str(e)][4] = idx_color[str(i)][4]
            if n>0 and len(bkk[n])>=boundary:
                similarn = (in_tuple(all_sims[i], n)[0] + in_tuple(all_sims[n], i)[0])/2
                if similarn > threshold:
                    idx_color[str(n)][4] = idx_color[str(i)][4]
            if s>0 and len(bkk[s])>=boundary:
                similars = (in_tuple(all_sims[i], s)[0] + in_tuple(all_sims[s], i)[0])/2
                if similars > threshold:
                    idx_color[str(s)][4] = idx_color[str(i)][4]
    return idx_color

In [53]:
def print_areas(idx_color):
    areas = set()
    for i in range(10):
        s = ""
        for j in range(10):
            idx = str((i*10)+j)
            if int(idx_color[idx][4]) < 10 and int(idx_color[idx][4]) >= 0:
                s += " "
            s += str(idx_color[idx][4]) +" "
            areas.add(idx_color[idx][4])
        print(s)
    print("\n #areas :" , len(areas))

In [54]:
def count_areas(idx_color):
    areas = set()
    for i in range(10):
        for j in range(10):
            areas.add(idx_color[str((i*10)+j)][4])
    return len(areas)

In [55]:
idx_color2 = copy.deepcopy(idx_color)

In [56]:
print_areas(idx_color)

 0  1  2  3  4  5  6  7  8  9 
10 11 12 13 14 15 16 17 18 19 
20 21 22 23 24 25 26 27 28 29 
30 31 32 33 34 35 36 37 38 39 
40 41 42 43 44 45 46 47 48 49 
50 51 52 53 54 55 56 57 58 59 
60 61 62 63 64 65 66 67 68 69 
70 71 72 73 74 75 76 77 78 79 
80 81 82 83 84 85 86 87 88 89 
90 91 92 93 94 95 96 97 98 99 

 #areas : 100


In [57]:
print_areas(idx_color2)

 0  1  2  3  4  5  6  7  8  9 
10 11 12 13 14 15 16 17 18 19 
20 21 22 23 24 25 26 27 28 29 
30 31 32 33 34 35 36 37 38 39 
40 41 42 43 44 45 46 47 48 49 
50 51 52 53 54 55 56 57 58 59 
60 61 62 63 64 65 66 67 68 69 
70 71 72 73 74 75 76 77 78 79 
80 81 82 83 84 85 86 87 88 89 
90 91 92 93 94 95 96 97 98 99 

 #areas : 100


In [58]:
idx_color2 = merge_area(0.7, idx_color)

In [59]:
print_areas(idx_color2)

-1 -1 -1  3 -1  5  6  7  8  9 
-1 -1 -1 13 14 15 16 17  8  9 
-1 -1 -1 23 24 25 -1 -1 -1 -1 
-1 -1 -1 33 34 35 36 -1 -1 39 
-1 -1 -1 43 44 45 46 47 48 -1 
-1 51 52 53 54 55 56 57 58 59 
-1 -1 -1 63 64 65 -1 -1 -1 69 
-1 -1 -1 73 64 75 76 -1 -1 79 
-1 -1 -1 83 84 85 86 87 88 -1 
-1 -1 -1 93 94 95 96 -1 98 99 

 #areas : 55


In [60]:
threshold = 0.7
areas = count_areas(idx_color2)
while areas > 11:
    idx_color2 = merge_area(threshold, idx_color2)
    threshold -= 0.01
    areas = count_areas(idx_color2)
    print(threshold)

0.69
0.6799999999999999
0.6699999999999999
0.6599999999999999
0.6499999999999999
0.6399999999999999
0.6299999999999999
0.6199999999999999
0.6099999999999999
0.5999999999999999
0.5899999999999999
0.5799999999999998
0.5699999999999998
0.5599999999999998
0.5499999999999998
0.5399999999999998
0.5299999999999998
0.5199999999999998
0.5099999999999998
0.4999999999999998
0.48999999999999977
0.47999999999999976
0.46999999999999975
0.45999999999999974
0.44999999999999973
0.4399999999999997
0.4299999999999997
0.4199999999999997
0.4099999999999997
0.3999999999999997
0.3899999999999997
0.37999999999999967
0.36999999999999966
0.35999999999999965
0.34999999999999964
0.33999999999999964
0.3299999999999996
0.3199999999999996
0.3099999999999996


In [61]:
areas

11

In [62]:
print_areas(idx_color2)

-1 -1 -1  3 -1  3  3  3  3  3 
-1 -1 -1  3  3  3  3  3  3  3 
-1 -1 -1  3  3  3 -1 -1 -1 -1 
-1 -1 -1  3  3  3  3 -1 -1 39 
-1 -1 -1 52 52  3  3 47 47 -1 
-1 51 52 52 52 52 56 47 47 47 
-1 -1 -1 52 52 52 -1 -1 -1 47 
-1 -1 -1 52 52 52 76 -1 -1 79 
-1 -1 -1 52 52 52 94 87 87 -1 
-1 -1 -1 52 94 94 94 -1 87 87 

 #areas : 11


In [512]:
print_areas(idx_color2)

-1 -1 -1  3 -1  5  5  5  5  5 
-1 -1 -1  3  3  3  5  5  5  5 
-1 -1 -1  3  3  3 -1 -1 -1 -1 
-1 -1 -1  3  3  3  3 -1 -1 39 
-1 -1 -1 51  3 45  3  3  3 -1 
-1 51 51 51 51 45  3  3  3  3 
-1 -1 -1 51 73 51 -1 -1 -1  3 
-1 -1 -1 73 73 73 76 -1 -1  3 
-1 -1 -1 73 93 93 93 93 88 -1 
-1 -1 -1 93 93 93 93 -1 88 99 

 #areas : 11


In [513]:
areas = set()
for i in range(10):
    s = ""
    for j in range(10):
        idx = (i*10)+j
        if len(bkk[idx]) < 99 and len(bkk[idx]) >= 0:
            s += " "
            if len(bkk[idx]) <10:
                s+= " "
        s += str(len(bkk[idx])) +" "
        #areas.add(idx_color[idx][4])
    print(s)
#print("\n #areas :" , len(areas))

  0   9   6  27   3  18 100  30  24  14 
  2   7   5  32  11  15  81 146  23  21 
  0   8   3  24  15  75   8   6   9   6 
  1   5   5  41  28  42  11   5   6  11 
  1   1   8  91  81 592 403  35  40   6 
  5  20  15  17 108 173 718 161  27  33 
  1   1   0  10  22 112   5   1   6  19 
  0   0   2  69  24 268  49   6   2  19 
  1   1   0  14 176  47  41  88 129   7 
  1   0   6  32  59  13  12   2  10  10 


In [514]:
idx_color["0"]

[(13.68578, 100.484605),
 (13.68578, 100.494605),
 (13.69578, 100.484605),
 (13.69578, 100.494605),
 '-1']